In [1]:
import numpy as np
from PIL import Image
import os
import cv2
from imagetype_classification import imagetype_classification

In [ ]:
#imagesのpre-process
TRAIN_PATH = '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train/'
DST_PATH =  '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train_default/'

SIZE = (640, 640)
for dirname in os.listdir(TRAIN_PATH):
        im = Image.open(TRAIN_PATH + dirname + '/images/' + dirname + '.png')
        im.save(DST_PATH + 'images/' + dirname + '.png', "png")

In [ ]:
#imagesのpre-process（白黒のみとcolorを分ける）
TRAIN_PATH = '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train/'
GRAY_PATH =  '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train_default_gray/'
COLOR_PATH =  '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train_default_color/'
THRESH = 10

for dirname in os.listdir(TRAIN_PATH):
        im = Image.open(TRAIN_PATH + dirname + '/images/' + dirname + '.png')
        im_array = np.asarray(im)
        if (im_array[:, :, 1] - im_array[:, :, 2]).sum() ** 2 <  THRESH:
            im.save(GRAY_PATH + 'images/' + dirname + '.png', "png")
        else:
            im.save(COLOR_PATH + 'images/' + dirname + '.png', "png")

In [ ]:
#maskのpre-process
TRAIN_PATH = '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train/'
DST_PATH =  '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train_default/'

#SIZE = (640, 640)
for dirname in os.listdir(TRAIN_PATH):
        # 空の行列を作るために、最初の1個だけ見ておく
        filename_one = os.listdir(TRAIN_PATH + dirname + '/masks/')[0]
        im = Image.open(TRAIN_PATH + dirname + '/masks/' + filename_one)
        size = im.size
        tmp = np.full((size[1], size[0]), False, dtype=bool)
        
        for filename in os.listdir(TRAIN_PATH + dirname + '/masks/'):
            im = Image.open(TRAIN_PATH + dirname + '/masks/' + filename)
            im = np.asarray(im)
            im = (im == 255)
            tmp = np.logical_or(tmp, im)
        im = tmp.astype(np.uint8) * 255
        im = Image.fromarray(im)
        im.save(DST_PATH + 'masks/' + dirname + '.png', "png")

In [ ]:
#test imagesのpre-process
TEST_PATH = '/data/unagi0/kanayama/dataset/nuclei_images/stage1_test/'
DST_PATH =  '/data/unagi0/kanayama/dataset/nuclei_images/stage1_test_preprocessed/'

SIZE = (640, 640)
for dirname in os.listdir(TEST_PATH):
        im = Image.open(TEST_PATH + dirname + '/images/' + dirname + '.png')
        im = im.resize(SIZE)
        im.save(DST_PATH + 'images/' + dirname + '.png', "png")

In [ ]:
# maskのpre-process（膨張処理により、わずかに輪郭を描く）
TRAIN_PATH = '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train/'
DST_PATH =  '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train_default/'

for dirname in os.listdir(TRAIN_PATH):
        # 空の行列を作るために、最初の1個だけ見ておく
        filename_one = os.listdir(TRAIN_PATH + dirname + '/masks/')[0]
        im = Image.open(TRAIN_PATH + dirname + '/masks/' + filename_one)
        size = im.size
        tmp = np.full((size[1], size[0]), False, dtype=bool)
        
        for filename in os.listdir(TRAIN_PATH + dirname + '/masks/'):
            im = Image.open(TRAIN_PATH + dirname + '/masks/' + filename)
            im = np.asarray(im)
            # 微妙に膨張させる
            im_large = cv2.dilate(im, np.ones((5,5),np.uint8), iterations = 1)
            im_large_bool = (im_large == 0)
            im_bool = (im == 255)            
            tmp = np.logical_and(im_large_bool, tmp)
            tmp = np.logical_or(tmp, im_bool)

        im = tmp.astype(np.uint8) * 255
        im = Image.fromarray(im)
        im.save(DST_PATH + 'masks/' + dirname + '.png', "png")

In [ ]:
# 染色方法ごとに画像を分類して保存（train）
IMAGE_PATH =  '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train_default/images/'
DST_PATH = '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train_default'
for image_name in os.listdir(IMAGE_PATH):
    imagetype = imagetype_classification(IMAGE_PATH + image_name)
    im = Image.open(IMAGE_PATH + image_name)
    
    file_path = DST_PATH + str(imagetype) + '/images/'
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    im.save(file_path + image_name)

In [ ]:
# edge の抽出
TRAIN_PATH = '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train/'
DST_PATH = '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train_default/'

for dirname in os.listdir(TRAIN_PATH):
        # 空の行列を作るために、最初の1個だけ見ておく
        filename_one = os.listdir(TRAIN_PATH + dirname + '/masks/')[0]
        im = Image.open(TRAIN_PATH + dirname + '/masks/' + filename_one)
        size = im.size
        tmp = np.full((size[1], size[0]), False, dtype=bool)
        
        for filename in os.listdir(TRAIN_PATH + dirname + '/masks/'):
            im = Image.open(TRAIN_PATH + dirname + '/masks/' + filename)
            im = np.asarray(im)
            # 微妙に膨張させる
            im_large = cv2.dilate(im, np.ones((5,5),np.uint8), iterations = 1)
            im_large_bool = (im_large == 255)
            im_bool = (im == 0)            
            tmp = np.logical_or(im_large_bool, tmp)
            tmp = np.logical_and(tmp, im_bool)

        im = tmp.astype(np.uint8) * 255
        im = Image.fromarray(im)
        im.save(DST_PATH + 'edges/' + dirname + '.png', "png")

In [60]:
#imagesのpre-process（白黒のみとcolorを分け、colorは上手くモノクロに変換して入れる。）
TRAIN_PATH = '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train/'
GRAY_PATH =  '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train_default_gray/'
THRESH = 10

for dirname in os.listdir(TRAIN_PATH):
        im = Image.open(TRAIN_PATH + dirname + '/images/' + dirname + '.png')
        im_array = np.asarray(im)
        if (im_array[:, :, 1] - im_array[:, :, 2]).sum() ** 2 <  THRESH:  # 白黒
            im.save(GRAY_PATH + 'images/' + dirname + '.png', "png")
        else:  # カラー
            im_gray = (im_array[:, :, 0] / 2 + im_array[:, :, 1] / 4 + im_array[:, :, 2] / 4)  # 赤に少し重み付けしてモノクロ化
            im_gray = np.broadcast_to(im_gray, (3, im_gray.shape[0], im_gray.shape[1]))
            im_gray = 255 - im_gray.transpose((1, 2, 0)).astype(np.uint8)  # 白黒反転
            im_gray = Image.fromarray(im_gray)
            im_gray.save(GRAY_PATH + 'images/' + dirname + '.png', "png")

In [13]:
#imagesのpre-process（白黒のみとcolorを分け、白黒は反転させて入れる。）
TRAIN_PATH = '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train/'
COLOR_PATH =  '/data/unagi0/kanayama/dataset/nuclei_images/stage1_train_default_color/'
THRESH = 10

for dirname in os.listdir(TRAIN_PATH):
        im = Image.open(TRAIN_PATH + dirname + '/images/' + dirname + '.png')
        im_array = np.asarray(im)
        if (im_array[:, :, 1] - im_array[:, :, 2]).sum() ** 2 <  THRESH:
            im_reverse_array = 255 - im_array[:, :, :3]  # alpha channelを除く
            im_reverse = Image.fromarray(im_reverse_array)
            im_reverse.save(COLOR_PATH + 'images/' + dirname + '.png', "png")
        else:
            im.save(COLOR_PATH + 'images/' + dirname + '.png', "png")